# What is about 

Load and EDA for scRNA-seq data.



In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Import and install modules 

In [ ]:
import time
t0start = time.time()

import pandas as pd
import numpy as np
import os
import sys

import matplotlib.pyplot as plt
import matplotlib as mpl
mpl.rcParams['figure.dpi'] = 70
plt.style.use('dark_background')

import seaborn as sns

from sklearn.decomposition import PCA

In [ ]:
!pip install scanpy
import scanpy as sc
import anndata

!pip install loompy # scanpy needs it to load loom files 

# Load data , show some info 

In [ ]:
#fn = '/kaggle/input/single-cell-rnaseq-data-related-to-cell-cycle/u2os.loom'
#fn = '/kaggle/input/single-cell-rna-sequencing-data-scc9-cell-line/SCC9_UPPER_AERODIGESTIVE_TRACT_proc.h5ad'
#fn = '/kaggle/input/single-cell-rna-sequencing-data-scc9-cell-line/SCC9_UPPER_AERODIGESTIVE_TRACT_raw.h5ad'
#fn = '/kaggle/input/scrna-d085t8-mouse-retina-tumor/D085T8.loom'
#fn = '/kaggle/input/scrna-d085t8-mouse-retina-tumor/D085T8.txt'
#path = '/kaggle/input/human-glioblastoma-dataset/' # matrix.mtx
#path = '/kaggle/input/scrnaseq-early-developing-mouse-retina-cells/'
#fn = '/kaggle/input/scrna-d085t8-mouse-retina-tumor/D085T8.txt'
#fn = '/kaggle/input/scrnaseq-early-developing-mouse-retina-cells/raw_count_matrix.mtx'
fn = '/kaggle/input/rds-scrnaseq-data/scrnaseq_Klein_mouse_embryo.csv'
str_data_inf = ' Klein mouse ESC ' # set information string on data - will be used in titles of plots etc 

import time
t0 = time.time()
if fn.endswith('.mtx') :
    #adata = sc.read_10x_mtx(path)
    adata = sc.read_mtx(fn)
    
    f = '/kaggle/input/scrnaseq-early-developing-mouse-retina-cells/genes.csv'
    df = pd.read_csv(f)
    print(df.shape)
    list_genes_upper = [ t.upper() for t in df['gene'].values ]
    adata.var = pd.DataFrame(index = list_genes_upper)# df['gene']) 
    #adata.var

elif fn.endswith('.loom') :
    adata = sc.read_loom(fn)
elif fn.endswith('.h5ad') :
    adata = sc.read_h5ad(fn)
elif fn.endswith('.csv'):
    df = pd.read_csv(fn,  index_col = 0) # 
    df = df.T
    display(df)
    var = pd.DataFrame(index = df.columns)
    obs = pd.DataFrame(index = df.index )
    adata =  anndata.AnnData(df.values,  obs=obs, var=var ) #
elif fn.endswith('.txt'):
    df = pd.read_csv(fn, sep = '\t', index_col = 0)
    df = df.T
    display(df)
    var = pd.DataFrame(index = df.columns)
    obs = pd.DataFrame(index = df.index )
    adata =  anndata.AnnData(df.values,  obs=obs, 
                         var=var ) #
    #print('New non-sparse adata: ', adata)

# For simplicity mouse genes - > human genes 
list_genes_upper = [t.upper() for t in adata.var.index ]
adata.var.index = list_genes_upper
    
print('Data loaded. ', np.round(time.time() - t0,1), 'seconds passed')
if ('obs' in dir(adata) ) and ('n.umi' not in adata.obs.columns):
    if ('obs' in dir(adata) ) and ('TotalUMIs' in  adata.obs.columns):
        adata.obs['n.umi'] = adata.obs['TotalUMIs'] 
    elif np.sum(adata.X) == int( np.sum(adata.X) ) :
        adata.obs['n.umi'] = adata.X.sum(axis = 1) 


adata_orig = adata.copy() 
print(np.sum(adata.X))
adata

In [ ]:
# Show some info on loaded data 
print('Data type: ', type(adata.X))
print('First elements of the count matrix:')
import scipy
if scipy.sparse.issparse(adata.X):
    print(adata.X[:5,:5].toarray())
else:
    print(adata.X[:5,:5])

if 'obs' in dir(adata):
    print()
    print('adata.obs.head(3):')
    display( adata.obs.head(3))
if 'var' in dir(adata):
    print()
    print('adata.var.head(3):')
    display( adata.var.head(3))

print()
print('adata')
print(adata)

# EDA

## Expression per cell

In [ ]:
fig = plt.figure(figsize = (20,6)); c = 0
c+=1; fig.add_subplot(1,2,c);
v = np.asarray(adata.X.sum(axis = 1)).ravel() # adata.obs['n.umi']
plt.plot(np.sort(v )) 
plt.title('Expression per cell')
plt.xlabel('cells sorted')
plt.ylabel('Expression Total')

c+=1; fig.add_subplot(1,2,c);
plt.hist(np.sort(v), bins = 100) 
plt.title('Expression per cell')

plt.show()

pd.Series(v).describe()


fig = plt.figure(figsize = (20,6)); c = 0
c+=1; fig.add_subplot(1,2,c);
v = np.asarray((adata.X != 0).sum(axis = 1)).ravel() # adata.obs['n.umi']
plt.plot(np.sort(v )) 
plt.title('Count genes expressed per cell')
plt.xlabel('cells sorted')
plt.ylabel('Count expressed genes')

c+=1; fig.add_subplot(1,2,c);
plt.hist(np.sort(v), bins = 100) 
plt.title('Count genes expressed per cell')

plt.show()

pd.Series(v).describe()



## Expression per gene

In [ ]:
v = np.asarray(adata.X.sum(axis = 0)).ravel()
display(pd.Series(v).describe())
v = np.log10(1+v)
fig = plt.figure(figsize = (20,6)); c = 0
c+=1; fig.add_subplot(1,2,c);
plt.plot(np.sort(v)) 
plt.title('LOG10 Expression per gene')
plt.xlabel('genes sorted')
plt.ylabel('Log10 (1+Counts) ')

c+=1; fig.add_subplot(1,2,c);
plt.hist(np.sort(v), bins = 30) 
plt.title('LOG10 Expression per gene')

plt.show()

pd.Series(v).describe()

## Show to expressed genes

In [ ]:
v = np.asarray(adata.X.sum(axis = 0)).ravel()
adata.var['Expression Sum'] = v
v = np.asarray( (adata.X != 0) .sum(axis = 0)).ravel()
adata.var['Expressed Cell'] = v
adata.var.sort_values('Expression Sum',ascending = False ).head(30) # ['counts']


## Correlation analysis

In [ ]:
number_top_genes_to_consider = 10_000
v = np.asarray(adata.X.sum(axis = 0 ) ).ravel()
v.shape
I = np.argsort(v)
I = I[::-1]
I = I[:number_top_genes_to_consider]

mask = np.ones( adata.X.shape[0]).astype(bool) 
t0 = time.time()
if scipy.sparse.issparse(adata.X):
    corr_matr = np.corrcoef(adata[mask].X[:,I].toarray().T) # Hint - use numpy , pandas is MUCH SLOWER   (df.corr() )
else:
    corr_matr = np.corrcoef(adata[mask].X[:,I].T) # Hint - use numpy , pandas is MUCH SLOWER   (df.corr() )
print('Calculated correlation matrix. Shape: ',corr_matr.shape,'  seconds passed:' , np.round( time.time() - t0,1) )

print()
print(np.min(corr_matr ), 'minimal correlation' )
corr_matr_abs = np.abs( corr_matr )
print(np.mean(corr_matr_abs ), 'average absolute correlation' )
print(np.median(corr_matr_abs), 'median absolute correlation' )
print(np.min(corr_matr_abs ), 'min absolute correlation' )
print(np.std(corr_matr_abs ), 'std absolute correlation' )
print()

v = corr_matr.flatten()
plt.figure(figsize=(14,8))
t0 = time.time()
plt.hist(v, bins = 50)
plt.title('correlation coefficients distribution')
plt.show()
print(time.time() - t0, 'seconds passed')

print(np.min(corr_matr ), 'minimal correlation' )
print(np.mean(corr_matr_abs ), 'average absolute correlation' )
print(np.median(corr_matr_abs), 'median absolute correlation' )
print(np.min(corr_matr_abs ), 'min absolute correlation' )
print(np.std(corr_matr_abs ), 'std absolute correlation' )
for t in [0.5,0.6, 0.7,0.8,0.9,0.95,0.97,0.98,.99]:
    print( ((np.abs(v) < 0.99999999) & (np.abs(v) > t)).sum()/2 , 'number of pairs correlated more than', t  )
v.shape

print()
print()
for threshold_corr in [0.99,0.98,00.97,0.95,0.93,0.90,0.85,0.8,0.75,0.7,0.65,0.6,0.55,.5,0.45,0.4,0.35,0.3]: 
    a,b = np.where( np.triu(np.abs(corr_matr),1) > threshold_corr )
    if len(a) > 50: break
print('for threshold:', threshold_corr,'we have ',len(a),' correlated pairs' )


d = pd.DataFrame()
for i in range(len(a)):
    Iai, Ibi =  I[a[i]], I[b[i]]
    d.loc[i,'Gene1'] = adata[mask].var.index[Iai] 
    d.loc[i,'Gene2'] = adata[mask].var.index[Ibi ] 
    d.loc[i,'Correlation'] = corr_matr[a[i],b[i]] 
    d.loc[i,'Correlation Abs'] = np.abs(d.loc[i,'Correlation'] )
    d.loc[i,'Gene1 Expression Sum'] = np.asarray(adata[mask].X[:,Iai].sum(axis = 0)).ravel()[0]
    d.loc[i,'Gene2 Expression Sum'] = np.asarray(adata[mask].X[:,Ibi].sum(axis = 0)).ravel()[0]
    
print(time.time() - t0, 'seconds passed')
    
d.sort_values('Correlation Abs',ascending = False).head(30)

In [ ]:
#clustermap
import seaborn as sns
t0 = time.time()
sns.clustermap(np.abs(corr_matr[:3000,:3000]),cmap='vlag');
print( np.round(time.time()- t0,1), ' seconds passed.')

In [ ]:
t0=time.time()
w,v = np.linalg.eig(corr_matr[:2000,:2000])
print(time.time()-t0, 'seconds passed')
w = np.real(w.ravel())
w = np.sort(w)[::-1]
plt.plot(w,'*-')
plt.title('Eigenvalues of correlation matrix ')
plt.show()

# Visualizations

## PCA

In [ ]:
import scipy
from sklearn.decomposition import TruncatedSVD
from sklearn.decomposition import PCA
import time
t0 = time.time()
if scipy.sparse.issparse(adata.X):
    reducer = TruncatedSVD(n_components=2, n_iter=7, random_state=42)
else:
    reducer = PCA(n_components=2)
    
r = reducer.fit_transform(adata.X)

fig = plt.figure(figsize = (25,12))
plt.title(str_data_inf + ' PCA  n_cells: ' + str(adata.X.shape[0]) +\
            ' n_genes: ' + str(adata.X.shape[1])  + ' '  , fontsize = 20   )#' 
if 'n.umi' not in adata.obs.columns: 
    ax = sns.scatterplot(x=v1, y = v2)#,  hue= color_by)#, palette = color_palette )#, palette = "viridis")# sns.color_palette("viridis", as_cmap=True),
                        #)# ,   alpha = 0.8, marker = '.')#, )#, legend=None)
else:
    color_by = adata.obs['n.umi'].values
    ax = sns.scatterplot(x=r[:,0], y = r[:,1],  hue= color_by)#, palette = color_palette )#, palette = "viridis")# sns.color_palette("viridis", as_cmap=True),
                        #)# ,   alpha = 0.8, marker = '.')#, )#, legend=None)
    plt.setp(ax.get_legend().get_texts(), fontsize='20') # for legend text
    plt.setp(ax.get_legend().get_title(), fontsize='20') # for legend title
plt.xlabel('PCA1' , fontsize = 20 )
plt.ylabel('PCA2' , fontsize = 20 )
plt.show()
print( np.round( time.time() - t0,1) , ' seconds passed ' ) 

## UMAP

In [ ]:
import scipy
import umap 
t0 = time.time()
reducer = umap.UMAP()
r = reducer.fit_transform(adata.X)

fig = plt.figure(figsize = (25,12))
plt.title(str_data_inf + ' UMAP n_cells: ' + str(adata.X.shape[0]) +\
            ' n_genes: ' + str(adata.X.shape[1])  + ' '  , fontsize = 20   )#' 
if 'n.umi' not in adata.obs.columns: 
    ax = sns.scatterplot(x=v1, y = v2)#,  hue= color_by)#, palette = color_palette )#, palette = "viridis")# sns.color_palette("viridis", as_cmap=True),
                        #)# ,   alpha = 0.8, marker = '.')#, )#, legend=None)
else:
    color_by = adata.obs['n.umi'].values
    ax = sns.scatterplot(x=r[:,0], y = r[:,1],  hue= color_by)#, palette = color_palette )#, palette = "viridis")# sns.color_palette("viridis", as_cmap=True),
                        #)# ,   alpha = 0.8, marker = '.')#, )#, legend=None)
    plt.setp(ax.get_legend().get_texts(), fontsize='20') # for legend text
    plt.setp(ax.get_legend().get_title(), fontsize='20') # for legend title

plt.xlabel('UMAP1' , fontsize = 20 )
plt.ylabel('UMAP2' , fontsize = 20 )
plt.show()
print( np.round( time.time() - t0,1) , ' seconds passed ' ) 

## Different axes PCA colored by genes

In [ ]:
plot_mode = 'PCA'# 'Phase_plot':
n_x_subplots = 3
genes_processing_mode = 'discretize_3bins_'# 'median_binarize_'

mask = np.ones( adata.X.shape[0]).astype(bool) # 

import scipy
from sklearn.decomposition import TruncatedSVD
from sklearn.decomposition import PCA
import time
t0 = time.time()
if scipy.sparse.issparse(adata.X):
    reducer = TruncatedSVD(n_components=10, n_iter=7, random_state=42)
else:
    reducer = PCA(n_components=10)
    
r = reducer.fit_transform(adata.X)

c = 0
for i,j in [(0,1),(2,3),(3,4),(5,6)]:#,(1,3),(2,3),(3,4)]:
    for color_by_mode in  ['n.umi',#'cell_cycle_phase', #  'PCA1ALL', 'n.umi', 'threshold_binarize_5_PCA1ALL',
        'E2F1','FOXM1']:#,'PCNA','TOP2A', 'CCNE1', 'CCNE2', 'CDK2', 'CCNB1','CCNB2','CCNB3','CCNA2',  ]: # , 'pct_counts_mt']:
        if c % n_x_subplots == 0:
            fig = plt.figure(figsize = (20,5) ); c = 0
            plt.suptitle(str_data_inf + ' PCA  n_cells: ' + str(adata.X.shape[0]) +\
                ' n_genes: ' + str(adata.X.shape[1])  + ' '  , fontsize = 20   )#' 
            c = 0
        c += 1; fig.add_subplot(1,n_x_subplots ,c)
        plt.title(str(color_by_mode) + ' PCA '+str(i+1)+','+str(j+1), fontsize = 20)

        color_by_field_name = color_by_mode
        if color_by_mode in adata.obs:
            color_by = (adata[mask].obs[color_by_field_name])
        elif color_by_mode in adata.var.index:
            I_gene = np.where(adata.var.index == color_by_mode)[0]
            v = adata[mask].X[:,I_gene]
            if scipy.sparse.issparse(adata.X):
                v = np.asarray(v.toarray()).ravel()
            if genes_processing_mode == 'median_binarize_':
                median_loc = np.median( v  )
                color_by = np.asarray (v  > median_loc ).ravel()
            elif genes_processing_mode == 'discretize_3bins_':
                t1 = np.percentile(v,33)    
                t2 = np.percentile(v,66)
                color_by = np.asarray (v  > t1 ).ravel().astype(int) + np.asarray (v  > t2 ).ravel().astype(int)
            else:
                color_by = v
        elif 'median_binarize_' in color_by_mode:
            color_by_field_name = color_by_mode[16:]
            if color_by_field_name in adata.obs:
                color_by = (adata[mask].obs[color_by_field_name]) > np.median( (adata.obs[color_by_field_name]) )
            else: color_by_mode = None
        elif 'threshold_binarize_' in color_by_mode:
            color_by_field_name = color_by_mode.split('_')[3]
            threshold_binarize = float( color_by_mode.split('_')[2] )
            if color_by_field_name in adata.obs:
                color_by = (adata[mask].obs[color_by_field_name]) > threshold_binarize
            else: color_by_mode = None
        else:
            color_by_mode = None

        v1 = r[:,i]
        v2 = r[:,j]
        if color_by_mode is None:
            ax = sns.scatterplot(x=v1, y = v2)# ,  hue= color_by,   alpha = 0.8, marker = '.')#, legend=None)
        else:
            if color_by_field_name == 'cell_cycle_phase':
                if len(np.unique(color_by)) == 3:
                    color_palette = ['red', 'green','blue' ]
                else:
                    color_palette = "viridis"# sns.color_palette("tab10")
            else:
                color_palette = "viridis"# sns.color_palette("tab10")        #color_by = (adata.obs[color_by_field_name]) > np.median( adata.obs[color_by_field_name].values ) 
            ax = sns.scatterplot(x=v1, y = v2,  hue= color_by, palette = color_palette )#, palette = "viridis")# sns.color_palette("viridis", as_cmap=True),
                                #)# ,   alpha = 0.8, marker = '.')#, )#, legend=None)
            plt.setp(ax.get_legend().get_texts(), fontsize='20') # for legend text
            plt.setp(ax.get_legend().get_title(), fontsize='20') # for legend title

        
plt.show()
print( np.round( time.time() - t0,1) , ' seconds passed ' ) 

## Plots in genes axes

In [ ]:
mask = np.ones( adata.X.shape[0]).astype(bool) # 

gene1 = 'E2F1' 
gene2 = 'FOXM1'
n_x_subplots = 4

c = 0 
for gene1,gene2 in [ ('E2F1','FOXM1'), ('PCNA','TOP2A'), ('CCNB1', 'CCNB2'), ('CCNE1', 'CCNB1'),
                   ('E2F1','PCNA'),('FOXM1','PCNA'), ('E2F1','TOP2A'),('FOXM1','TOP2A'), ]: 
    #,'PCNA','TOP2A', 'CCNE1', 'CCNE2', 'CDK2', 'CCNB1','CCNB2','CCNB3','CCNA2',  ]: # , 'pct_counts_mt']:
    if gene1 not in adata.var.index : continue
    if gene2 not in adata.var.index : continue
        
    color_by_mode = 'n.umi' # ,#'cell_cycle_phase', #  'PCA1ALL', 'n.umi', 'threshold_binarize_5_PCA1ALL',
    if color_by_mode in adata.obs:
        color_by_field_name = color_by_mode
        color_by = (adata[mask].obs[color_by_field_name])
    else:
        color_by_mode = None
    
    if c % n_x_subplots == 0:
        fig = plt.figure(figsize = (20,6) ); c = 0
        plt.suptitle(str_data_inf + ' Genes axes   n_cells: ' + str(adata.X.shape[0]) +\
            ' n_genes: ' + str(adata.X.shape[1])  + ' color by '+ str(color_by_mode) , fontsize = 20   )#' 
        c = 0
        
    c += 1; fig.add_subplot(1,n_x_subplots ,c)
    plt.title(gene1 + ' ' + gene2) # str(color_by_mode) + ' PCA '+str(i)+','+str(j))

    plt.legend()
    I_gene = np.where(adata.var.index == gene1)[0]
    v = adata[mask].X[:,I_gene]
    if scipy.sparse.issparse(adata.X):
        v = np.asarray(v.toarray()).ravel()
    else:
        v = np.asarray(v).ravel()
        
    v1 = v
    I_gene = np.where(adata.var.index == gene2)[0]
    v = adata[mask].X[:,I_gene]
    if scipy.sparse.issparse(adata.X):
        v = np.asarray(v.toarray()).ravel()
    else:
        v = np.asarray(v).ravel()
    v2 = v


    if color_by_mode is None:
        ax = sns.scatterplot(x=v1,y=v2)# ,   hue= color_by)#, alpha = 0.7, marker = '.', legend = None)
    else:
        ax = sns.scatterplot(x=v1,y=v2,   hue= color_by)#, alpha = 0.7, marker = '.', legend = None)
        plt.setp(ax.get_legend().get_texts(), fontsize='15') # for legend text
        plt.setp(ax.get_legend().get_title(), fontsize='15') # for legend title    plt.xlabel('PCA1' , fontsize = 20 )
        ax.legend().set_visible(False)

    plt.xlabel(gene1,  fontsize = 20)
    plt.ylabel(gene2,  fontsize = 20)
plt.show()  

# Preprocessing

## Auxilliary functions

In [ ]:
def _smooth_adata_by_pooling(adata, X_embed, n_neighbours=10, copy=False):
    # adata_pooled = adata.copy if copy else adata
    nbrs = NearestNeighbors(n_neighbors=n_neighbours).fit(X_embed)
    print( 'done: nbrs = NearestNeighbors(n_neighbors=n_neighbours).fit(X_embed) ')
    distances, indices = nbrs.kneighbors(X_embed)
    print( 'done: distances, indices = nbrs.kneighbors(X_embed) ')
    nd_array_X = _get_nd_array(adata.X)
    print('done: nd_array_X = _get_nd_array(adata.X)' )
    adata.X = _smooth_matrix_by_pooling(nd_array_X , indices)
    print( 'done: adata.X = _smooth_matrix_by_pooling(_get_nd_array(adata.X), indices) ')
    if "matrix" in adata.layers:
        adata.layers["matrix"] = _smooth_matrix_by_pooling(
            _get_nd_array(adata.layers["matrix"]), indices
        )
    if "spliced" in adata.layers:
        adata.layers["spliced"] = _smooth_matrix_by_pooling(
            _get_nd_array(adata.layers["spliced"]), indices
        )
    if "unspliced" in adata.layers:
        adata.layers["unspliced"] = _smooth_matrix_by_pooling(
            _get_nd_array(adata.layers["unspliced"]), indices
        )

    adata.uns["scycle"] = True


def _smooth_matrix_by_pooling(matrix, indices):
    matrix_pooled = matrix.copy()
    t0 = time.time()
    #print(indices.shape,  matrix.shape, type(matrix) )
    for i in range(len(indices)):
        #print(i,len(indices), time.time()-t0 )
        matrix_pooled[i, :] = np.mean(matrix[indices[i], :], axis=0)
    return matrix_pooled


def _get_nd_array(arr):
    x = None
    if str(type(arr)):
        x = arr
    else:
        print('start:  x = arr.toarray()')
        x = arr.toarray()
        print('finished:  x = arr.toarray()')
    return x

def calc_scores(anndata,signature_dict):
    matrix = anndata.to_df().to_numpy()
    scores_dic = {}
    for key in signature_dict:
        names = np.array(signature_dict[key])
        inds = np.where(np.isin(anndata.var_names,names))[0]
        matrix_sel = matrix[:,inds]
        scores = np.mean(matrix_sel,axis=1)
        scores_dic[key] = scores
    return scores_dic

def find_nonproliferative_cells(adata,estimation_fraction_nonproliferating_cells=0.3, 
                                number_of_nodes=30,max_number_of_iterations = 20,
                               number_of_sigmas=3.0,Mu=1.0):
    all_markers = list(adata.uns['S-phase_genes'])+list(adata.uns['G2-M_genes'])
    Xccm = adata[:,list(set(all_markers)&set(adata.var_names))].X
    cc_score = np.array(list(np.mean(Xccm,axis=1)))

    ind_sorted_prolif = np.argsort(cc_score)
    ind_nonprolif = ind_sorted_prolif[0:int(len(adata)*estimation_fraction_nonproliferating_cells)]
    adata.obs['proliferating'] = np.empty(len(adata)).astype(np.int)
    adata.obs['proliferating'][:] = 1
    adata.obs['proliferating'][ind_nonprolif] = 0
    
    sc.pl.scatter(adata,x='S-phase',y='G2-M',color='proliferating')

    fraction_nonprolif_old = estimation_fraction_nonproliferating_cells

    for i in range(max_number_of_iterations):
        X_elpigraph_training = adata.obs[['S-phase','G2-M']].to_numpy().astype(np.float64)
        u = X_elpigraph_training.copy()
        ind_prolif = np.where(np.array(adata.obs['proliferating'])==1)[0]        
        X_elpigraph_training = X_elpigraph_training[ind_prolif,:]

        egr = elpigraph.computeElasticPrincipalCircle(X_elpigraph_training,number_of_nodes,
                                                      Mu=Mu,drawPCAView=False,verbose=False)
        partition, dists = elpigraph.src.core.PartitionData(X = u, NodePositions = egr[0]['NodePositions'], 
                                                                MaxBlockSize = 100000000, TrimmingRadius = np.inf,
                                                                SquaredX = np.sum(u**2,axis=1,keepdims=1))

        mndist = np.mean(dists[ind_prolif])
        #plt.hist(dists,bins=100)
        #plt.show()
        #plt.hist(dists[ind_prolif],bins=100)
        #plt.show()        
        intervaldist = np.std(dists[ind_prolif])*number_of_sigmas
        tt1 = np.array(1-adata.obs['proliferating'])
        tt2 = np.zeros(len(tt1)).astype(np.int)
        for k,d in enumerate(dists):
            if d>mndist+intervaldist:
                tt2[k] = 1
        nonprolif_new = np.array(tt1) * np.array(tt2)
        adata.obs['proliferating'] = 1-nonprolif_new
        #adata.obs['dists'] = dists
        #sc.pl.scatter(adata,x='S-phase',y='G2-M',color='dists')
        fraction_nonprolif = 1-np.sum(adata.obs['proliferating'])/len(adata)
        print('\n\n===========\nIteration',i,'Fraction of non-proliferating cells:',fraction_nonprolif,'\n==============\n\n\n')    
        if np.abs(fraction_nonprolif-fraction_nonprolif_old)<0.01:
            break
        fraction_nonprolif_old = fraction_nonprolif
        
def _compute_principal_circle(adata,number_of_nodes=30,n_components = 30):
    #driving_feature = np.array(adata_orig[adata.uns['ind_samples']].obs['total_counts'])
    #driving_feature_weight = 1.0
    #egr, partition, X, Xp = _compute_principal_circle(adata,number_of_nodes=40,
    #                                                  driving_feature=driving_feature,driving_feature_weight=0.1)

    X = adata.X
    X_prolif = adata.X[np.where(adata.obs['proliferating']==1)[0],:]
    #driving_feature = driving_feature[adata.obs[np.where(adata.obs['proliferating']==1)[0]]]

    mn_prolif = np.mean(X_prolif,axis=0)
    pca = PCA(n_components=n_components)
    u = pca.fit_transform(X_prolif)
    v = pca.components_.T
    #X_pca = adata.obsm['X_pca'].astype(np.float64)
    X_elpigraph_training = u.astype(np.float64)
    #std = np.std(X_elpigraph_training)
    #driving_feature = stats.zscore(driving_feature)*std*driving_feature_weight
    #driving_feature = driving_feature.reshape(-1,1)
    #X_elpigraph_training = np.concatenate([X_elpigraph_training,driving_feature],axis=1)
    #print('std=',std)
    #print('std df=',np.std(driving_feature))

    #print(X_elpigraph_training.shape)
    egr = elpigraph.computeElasticPrincipalCircle(X_elpigraph_training,number_of_nodes,Mu=0.2)
    partition, dists = elpigraph.src.core.PartitionData(X = X_elpigraph_training, NodePositions = egr[0]['NodePositions'], 
                                                            MaxBlockSize = 100000000, TrimmingRadius = np.inf,
                                                            SquaredX = np.sum(X_elpigraph_training**2,axis=1,keepdims=1))
    Xp = (X-mn_prolif)@v
    #number_of_driving_features = driving_feature.shape[1]
    #node_positions_reduced = egr[0]['NodePositions'][:,:-number_of_driving_features]
    nodep = egr[0]['NodePositions']
    partition, dists = elpigraph.src.core.PartitionData(X = Xp, NodePositions = nodep, 
                                                           MaxBlockSize = 100000000, TrimmingRadius = np.inf,
                                                           SquaredX = np.sum(Xp**2,axis=1,keepdims=1))

    #nodep = egr[0]['NodePositions'][:,:-number_of_driving_features]
    #edges = egr[0]['Edges'][0]
    #X_elpigraph_training = X_elpigraph_training[:,:-number_of_driving_features]
    #egr[0]['NodePositions'] = nodep
    return egr, partition, X_elpigraph_training, Xp


def _compute_principal_curve(adata,number_of_nodes=30, Mu=0.3, Lambda=0.001,n_components=30):
    X = adata.X
    X_prolif = adata.X[np.where(adata.obs['proliferating']==1)[0],:]

    mn_prolif = np.mean(X_prolif,axis=0)
    pca = PCA(n_components=n_components)
    u = pca.fit_transform(X_prolif)
    v = pca.components_.T
    #X_pca = adata.obsm['X_pca'].astype(np.float64)
    
    adata.varm['pc_components_elpigraph'] = v
    adata.varm['mean_point_elpigraph'] = mn_prolif
    
    X_elpigraph_training = u.astype(np.float64)

    egr, starting_node = compute_principal_curve_from_circle(X_elpigraph_training,
                                                             n_nodes=number_of_nodes,
                                                            produceTree=False,Mu=Mu,Lambda=Lambda)

    partition_, dists = elpigraph.src.core.PartitionData(X = X_elpigraph_training, NodePositions = egr[0]['NodePositions'], 
                                                            MaxBlockSize = 100000000, TrimmingRadius = np.inf,
                                                            SquaredX = np.sum(X_elpigraph_training**2,axis=1,keepdims=1))
    Xp = (X-mn_prolif)@v
    partition_, dists = elpigraph.src.core.PartitionData(X = Xp, NodePositions = egr[0]['NodePositions'], 
                                                            MaxBlockSize = 100000000, TrimmingRadius = np.inf,
                                                            SquaredX = np.sum(Xp**2,axis=1,keepdims=1))

    nodep = egr[0]['NodePositions']
    edges = egr[0]['Edges'][0]
    return egr, partition_, starting_node,X_elpigraph_training, Xp

def compute_principal_curve_from_circle(X,n_nodes=30,Mu=0.1,Lambda=0.01, produceTree=False):
    egr = elpigraph.computeElasticPrincipalCircle(X,int(n_nodes/2),Mu=Mu)
    nodep = egr[0]['NodePositions']
    edges = egr[0]['Edges'][0]
    partition_, dists = elpigraph.src.core.PartitionData(X = X, NodePositions = nodep, 
                                                            MaxBlockSize = 100000000, TrimmingRadius = np.inf,
                                                            SquaredX = np.sum(X**2,axis=1,keepdims=1))
    node_sizes = np.array([len(np.where(partition_==i)[0]) for i in range(nodep.shape[0])])
    node_min = np.argmin(node_sizes)
    edges_open = edges.copy()
    k = 0
    starting_node = -1
    while k<edges_open.shape[0]:
        e = edges_open[k,:]
        if (e[0]==node_min)|(e[1]==node_min):
            edges_open = np.delete(edges_open,k,axis=0)
            if e[0]==node_min:
                starting_node = e[1]
            if e[1]==node_min:
                starting_node = e[0]
        else:
            k = k+1
    nodep_open = np.delete(nodep,node_min,axis=0)
    if starting_node>node_min:
        starting_node = starting_node-1
    for e in edges_open:
        if e[0]>node_min:
            e[0]=e[0]-1
        if e[1]>node_min:
            e[1]=e[1]-1
    if produceTree:
        egrl = elpigraph.computeElasticPrincipalTree(X,n_nodes,InitNodePositions=nodep_open,
                                                 InitEdges=edges_open,Lambda=Lambda,Mu=Mu,
                                                 alpha=0.01,FinalEnergy='Penalized')
    else:
        egrl = elpigraph.computeElasticPrincipalCurve(X,n_nodes,InitNodePositions=nodep_open,
                                                 InitEdges=edges_open,Lambda=Lambda,Mu=Mu,
                                                 alpha=0.01,FinalEnergy='Penalized')
    return egrl, starting_node

def subtract_cell_cycle_trajectory(X,partition):
    points = range(X.shape[0])
    r2scores = []
    X1 = X[points,:]
    X_ro = np.zeros((X.shape[0],X.shape[1]))
    partition_points = partition[points]
    inds = {}
    for k in range(len(partition_points)):
        j = partition_points[k][0]
        if not j in inds:
            inds[j] = [k]
        else:
            inds[j].append(k)
    XT = X1.T
    for j in range(X1.shape[0]):
        k = partition_points[j][0]
        ind = np.array(inds[k])
        X_ro[j,:] = (XT[:,j] - np.mean(XT[:,ind],axis=1)).T
    residue_matrix = X1 - X_ro
    residues_var = np.var(residue_matrix,axis=0)
    vrs = np.var(X1,axis=0)
    r2scores = residues_var/vrs
    return X_ro, residue_matrix, r2scores

In [ ]:
list_genes2include_mandotory = ['E2F1','FOXM1', 'ESRRB', 'NR5A2',  'HIST1H4C', 'FOXO3' , 'EZH1',  'ANLN' ]
list_genes2include_mandotory += ['PCNA','TOP2A' ]

list_genes2include_mandotory += ['WEE1','WEE2'] # Inhibitor of CDK1 https://en.wikipedia.org/wiki/Wee1
list_genes2include_mandotory += ['CDK1', 'UBE2C','TOP2A','HIST1H4E','HIST1H4C'] # Hsiao2020GenomeResearch 5 genes 'HIST1H4E' = H4C5,'HIST1H4C'=H4C3
list_genes2include_mandotory +=  ['MYBL2', 'TP53', 'KLF4', 'CDKN1A', 'CDKN2A', 'RB1', 'MDM2', 'MDM4', 'ZBTB20', 'GAPDH',  'EFNA5', 'LINC00511' ]
# MYBL2 https://en.wikipedia.org/wiki/MYBL2 - transcription factor
# The protein encoded by this gene, a member of the MYB family of transcription factor genes, is a nuclear protein involved in cell cycle progression. The encoded protein is phosphorylated by cyclin A/cyclin-dependent kinase 2 during the S-phase of the cell cycle and possesses both activator and repressor activities.
# https://www.ncbi.nlm.nih.gov/pmc/articles/PMC6788831/
# Binding of FoxM1 to G2/M gene promoters is dependent upon B-Myb Christin F Down 1, Julie Millour, Eric W-F Lam, Roger J Watson
#https://pubmed.ncbi.nlm.nih.gov/23555285/
#Long noncoding RNA MALAT1 controls cell cycle progression by regulating the expression of oncogenic transcription factor B-MYB -

list_genes2include_mandotory += [ 'CCNE1', 'CCNE2', 'CDK2'] +  ['CCNB1','CCNB2','CCNB3'] + ['CCNA1',
'CCNA2','CCNF', 'CDK1'] + ['CCND1', 'CCND2', 'CCND3'] + [ 'CDK4','CDK6']
#                       'CCNA1','CCNA2', 'CCNE1', 'CCNE2',]
list_genes2include_mandotory += ['CDK11A', # All of the form: CDKnumber*  - 27
                                 'CDK14', 'CDK17', 'CDK13', 'CDK5RAP1', 'CDK16', 'CDK6', 'CDK5RAP3', 'CDK2AP1', 'CDK18', 'CDK2', 'CDK8', 'CDK7', 'CDK4', 'CDK9', 'CDK5RAP2', 'CDK15', 'CDK19', 'CDK20', 'CDK5', 'CDK12', 'CDK2AP2', 'CDK1', 'CDK5R2', 'CDK5R1', 'CDK10', 'CDK11B']
list_genes2include_mandotory += ['CDKN3', # 10 CDKN* - A cyclin-dependent kinase inhibitor protein is a protein which inhibits the enzyme cyclin-dependent kinase (CDK). 
         'CDKN1B', 'CDKN2C', 'CDKN1A', 'CDKN2D', 'CDKN1C', 'CDKN2A', 'CDKN2AIP', 'CDKN2AIPNL', 'CDKN2B-AS1']
# p16	CDKN2A	Cyclin-dependent kinase 4, Cyclin-dependent kinase 6
# p15	CDKN2B	Cyclin-dependent kinase 4
#p18	CDKN2C	Cyclin-dependent kinase 4, Cyclin-dependent kinase 6
#p19	CDKN2D	Cyclin-dependent kinase 4, Cyclin-dependent kinase 6
#p21 / WAF1	CDKN1A[2]	Cyclin E1/Cyclin-dependent kinase 2
#p27	CDKN1B	Cyclin D3/Cyclin-dependent kinase 4, Cyclin E1/Cyclin-dependent kinase 2
#p57	CDKN1C	Cyclin E1/Cyclin-dependent kinase 2
# https://en.wikipedia.org/wiki/Cyclin-dependent_kinase_inhibitor_protein
    
list_genes2include_mandotory += ['CDKL3', # All starts with CDK - 43 
                                 'CDKL5', 'CDK11A', 'CDK14', 'CDK17', 'CDK13', 'CDKL1', 'CDKN3', 'CDK5RAP1', 'CDK16', 'CDK6', 'CDK5RAP3', 'CDKN1B', 'CDK2AP1', 'CDK18', 'CDKN2C', 'CDK2', 'CDKN1A', 'CDKN2D', 'CDKN1C', 'CDK8', 'CDK7', 'CDK4', 'CDK9', 'CDK5RAP2', 'CDK15', 'CDKL2', 'CDKAL1', 'CDKN2A', 'CDK19', 'CDK20', 'CDK5', 'CDK12', 'CDK2AP2', 'CDKN2AIP', 'CDK1', 'CDK5R2', 'CDK5R1', 'CDK10', 'CDKL4', 'CDKN2AIPNL', 'CDKN2B-AS1', 'CDK11B']

list_genes2include_mandotory += ['CDC27', #  "cdc" in its name refers to "cell division cycle"
 'CDC42', 'CDC14A', 'CDC14B', 'CDC45', 'CDC6', 'CDC23', 'CDC5L', 'CDC7', 'CDC34', 'CDC25B', 'CDC37', 'CDC37L1', 'CDCA3', 'CDC20', 'CDC42EP1', 'CDC16', 'CDC73', 'CDCA8', 'CDC42BPA', 'CDCA7', 'CDCA5', 'CDC42EP2', 'CDC123', 'CDC25C', 'CDC42SE2', 'CDC42EP3', 'CDCP1', 'CDC25A', 'CDC20B', 'CDCA7L', 'CDC42EP5', 'CDC40', 'CDCA4', 'CDC42BPG', 'CDC26', 'CDC42EP4', 'CDCA2', 'CDC42SE1', 'CDC42BPB', 'CDC42-IT1', 'CDC20P1', 'CDC42P6', 'CDC27P2', 'CDC42P5', 'CDC37L1-AS1', 'CDC27P3']
# Cdc25 is a dual-specificity phosphatase first isolated from the yeast Schizosaccharomyces pombe as a cell cycle defective mutant.[2] As with other cell cycle proteins or genes such as Cdc2 and Cdc4, the "cdc" in its name refers to "cell division cycle". Dual-specificity phosphatases are considered a sub-class of protein tyrosine phosphatases. By removing inhibitory phosphate residues from target cyclin-dependent kinases (Cdks),[3] Cdc25 proteins control entry into and progression through various phases of the cell cycle, including mitosis and S ("Synthesis") phase.

list_genes2include_mandotory += ['E2F2', # 10 genes E2F transc. factors  
     'E2F1', 'E2F3', 'E2F8', 'E2F5', 'E2F7', 'E2F6', 'E2F3P2', 'E2F4', 'E2F3-IT1']

list_genes2include_mandotory += ['GATA1', 'FOXA1', 'BRD4', 'KMT2A' ]
# 'GATA1', 'FOXA1', 'BRD4', 'KMT2A' - transcription factors are retained in mitotic chromosomes (from paper "Trans landscape of the human cell cycle" )

list_histone_genes_from_wiki = ['H1F0' , 'H1FNT' , 'H1FOO' , 'H1FX' ,
'HIST1H1A' , 'HIST1H1B' , 'HIST1H1C' , 'HIST1H1D' , 'HIST1H1E' , 'HIST1H1T' ,
'H2AFB1' , 'H2AFB2' , 'H2AFB3' , 'H2AFJ' , 'H2AFV' , 'H2AFX' , 'H2AFY' , 'H2AFY2' , 'H2AFZ' ,
'HIST1H2AA' , 'HIST1H2AB' , 'HIST1H2AC' , 'HIST1H2AD' , 'HIST1H2AE' , 'HIST1H2AG' , 'HIST1H2AI' , 'HIST1H2AJ' , 'HIST1H2AK' , 'HIST1H2AL' , 'HIST1H2AM' ,
'HIST2H2AA3' , 'HIST2H2AC' ,
'H2BFM' , 'H2BFS' , 'H2BFWT' ,
'HIST1H2BA' , 'HIST1H2BB' , 'HIST1H2BC' , 'HIST1H2BD' , 'HIST1H2BE' , 'HIST1H2BF' , 'HIST1H2BG' , 'HIST1H2BH' , 'HIST1H2BI' ,
'HIST1H2BJ' , 'HIST1H2BK' , 'HIST1H2BL' , 'HIST1H2BM' , 'HIST1H2BN' , 'HIST1H2BO' ,
'HIST2H2BE' ,
'HIST1H3A' , 'HIST1H3B' , 'HIST1H3C' , 'HIST1H3D' , 'HIST1H3E' , 'HIST1H3F' , 'HIST1H3G' , 'HIST1H3H' , 'HIST1H3I' , 'HIST1H3J' ,
'HIST2H3C' ,
'HIST3H3' ,
'HIST1H4A' , 'HIST1H4B' , 'HIST1H4C' , 'HIST1H4D' , 'HIST1H4E' , 'HIST1H4F' , 'HIST1H4G' , 'HIST1H4H' , 'HIST1H4I' , 'HIST1H4J' , 'HIST1H4K' , 'HIST1H4L' ,
'HIST4H4' ]
list_genes2include_mandotory += list_histone_genes_from_wiki

In [ ]:
print(adata_orig.X.sum()) # 547605500.0 - csv #  327662140.0


In [ ]:
adata_orig.var_names_make_unique()
adata_orig.var_names_make_unique()

## Preprocessing and conversion from sparse adata.X to dense adata.X using only 10 000 top expressed genes


In [ ]:
from sklearn.neighbors import NearestNeighbors

# First standard preprocessing
# Then "pooling" - fighting with zeros 

# Params: 
n_top_genes_to_keep = 10000

threshold_pct_counts_mt = 40 
min_count = 1000 # Threshold on min total_counts
max_count = 10000# Threshold on max total_counts

if str_data_inf == ' D085T8 txtfile':
    threshold_pct_counts_mt = 4
    min_count = 500
    max_count = 20000# Threshold on max total_counts
elif str_data_inf == ' D085T8 loomfile':
    threshold_pct_counts_mt = 10
    min_count = 500
    max_count = 10000# Threshold on max total_counts
elif str_data_inf == ' sciPlex2_A549 ':
    threshold_pct_counts_mt = 15
    min_count = 500
elif str_data_inf == ' K562_HDACs+vehicle ':
    threshold_pct_counts_mt = 20
    min_count = 500
    max_count = 3000# 12000
elif 'Lundberg2020' in str_data_inf :
    if flag_load_from_loom_file:
        threshold_pct_counts_mt = 32
        min_count = 150_000
        max_count = 420_000# 12000
    threshold_pct_counts_mt = 25
    min_count = 300_000
    max_count = 700_000# 12000
    


print(adata_orig.X.sum())

# ################################################################################################
# Preprocessing first step: filter CELLs by  counts and level of MT-percent
# thresholds are set visually looking on violin plots 
# Examples:
# threshold_pct_counts_mt = 20 - 40  min_count = 500 - 1000; max_count = 10000 - 12000; 
# These thresholds depends on cell line dataset


adata = adata_orig.copy()
print(adata)

adata.var['mt'] = adata.var_names.str.startswith('MT-')  # annotate the group of mitochondrial genes as 'mt'
#sv.pp.remove_duplicate_cells(adata)
sc.pp.calculate_qc_metrics(adata, qc_vars=['mt'], percent_top=None, log1p=False, inplace=True)
sc.pl.violin(adata, ['n_genes_by_counts', 'n.umi', 'total_counts', 'pct_counts_mt'],jitter=1.9, multi_panel=True)
median_count = np.median(adata.obs['total_counts'])
print('Median total counts =',median_count)
print('min(n_genes_by_counts):',adata.obs['n_genes_by_counts'].min() )
print('min(total_counts):',adata.obs['total_counts'].min() )
# min_count = np.max((median_count/2,5000))


print('Thresholds: min_count=',min_count,'max_count=',max_count)

print('Look at total_counts va MT-percent, expect some linear dependence - but does not happen: ')
sc.pl.scatter(adata, x='total_counts', y='pct_counts_mt')

inds1 = np.where((adata.obs['total_counts']>min_count) & (adata.obs['total_counts']<max_count))
inds2 = np.where(adata.obs['pct_counts_mt']<threshold_pct_counts_mt)
print(len(inds1[0]),'samples pass the count filter')
print(len(inds2[0]),' samples pass the mt filter')
ind_samples = np.intersect1d(inds1[0],inds2[0])
print('Samples selected',len(ind_samples))
adata.uns['ind_samples'] = ind_samples

# Here we cut cells. Filtering out those with counts too low or too big
adata = adata[ind_samples,:]

# ################################################################################################
# Preprocessing second step: 
# 1) normalization to some value, i.e. median of the total counts
# 2) taking logs
# 3) keeping only higly variable genes

sc.pp.normalize_total(adata, target_sum=np.median(adata.obs["total_counts"]))
sc.pp.log1p(adata)
sc.pp.highly_variable_genes(adata,n_top_genes=n_top_genes_to_keep,n_bins=20)
ind_genes = np.where(adata.var['highly_variable'])[0]
ind_genes2 = np.where(adata.var.index.isin(list_genes2include_mandotory ) )[0]
ind_genes = list( set( ind_genes )|set( ind_genes2 ) )
adata = adata[:,ind_genes]

print('Violin plots after filtering cells and genes')
sc.pl.violin(adata, ['n_genes_by_counts','n.umi', 'total_counts', 'pct_counts_mt'],jitter=1.9, multi_panel=True)
# ################################################################################################
# Preprocessing third step: 
# Mainly: Create non-sparse adata (otherwise next step - pooling works extremely slow)
# 1) calculate PCA for adata previously calculated adata (which is cutted, normed, logged) - and keep it for use in pooling
# 2) create new non-sparse adata with cells and genes, obtained by filters on previous steps

#print('Check - we should see ints, otherwise data has been corrupted: ')
#print(adata_orig.X[:4,:5].toarray())

sc.tl.pca(adata,n_comps=30)
X_pca = adata.obsm['X_pca']
X_pca1 = X_pca.copy()

adata_orig[ind_samples,ind_genes].var.shape
if scipy.sparse.issparse(adata.X):
    XX2 = adata_orig[ind_samples,ind_genes].X.toarray()
else:
    XX2 = adata_orig[ind_samples,ind_genes].X.copy()
adata =  anndata.AnnData(XX2,  obs=adata_orig[ind_samples,ind_genes].obs.copy(), 
                     var=adata_orig[ind_samples,ind_genes].var.copy() ) #
print('New non-sparse adata: ', adata)

#sv.pp.remove_duplicate_cells(adata)
adata.var['mt'] = adata.var_names.str.startswith('MT-')  # annotate the group of mitochondrial genes as 'mt'
sc.pp.calculate_qc_metrics(adata, qc_vars=['mt'], percent_top=None, log1p=False, inplace=True)

print('Violin plots for new adata')
sc.pl.violin(adata, ['n_genes_by_counts', 'n.umi', 'total_counts', 'pct_counts_mt'],jitter=1.9, multi_panel=True)


# ################################################################################################
# Preprocessing 4-th step: may take quite long - 10 minutes for 40K celss 
# Mainly: pooling - we are making pooling on original counts, but calculate KNN graph using PCA obtained from normed/log data
# (that is why we forget adata obtained on the steps 1-2, and will do norm-log again here )
# 1) pooling
# 2) quality metrics calculations
# 3) normalization 
# 4) log

t0 = time.time()
_smooth_adata_by_pooling(adata,X_pca,n_neighbours=20)
print(time.time()-t0,'seconds passed')
sc.pp.calculate_qc_metrics(adata, percent_top=None, log1p=False, inplace=True)
sc.pp.normalize_total(adata, target_sum=np.median(adata.obs["total_counts"]))
sc.pp.log1p(adata)
#adata = adata[:,ind_genes]
adata.uns['ind_genes'] = ind_genes

sc.tl.pca(adata,n_comps=30)
X_pca = adata.obsm['X_pca']

adata

In [ ]:
type(adata.X)

# Correlation analysis

In [ ]:
number_top_genes_to_consider = 1_000
v = np.asarray(adata.X.sum(axis = 0 ) ).ravel()
v.shape
I = np.argsort(v)
I = I[::-1]
I = I[:number_top_genes_to_consider]
number_top_genes_to_consider = 10_000
v = np.asarray(adata.X.sum(axis = 0 ) ).ravel()
v.shape
I = np.argsort(v)
I = I[::-1]
I = I[:number_top_genes_to_consider]

mask = np.ones( adata.X.shape[0]).astype(bool) 
t0 = time.time()
if scipy.sparse.issparse(adata.X):
    corr_matr = np.corrcoef(adata[mask].X[:,I].toarray().T) # Hint - use numpy , pandas is MUCH SLOWER   (df.corr() )
else:
    corr_matr = np.corrcoef(adata[mask].X[:,I].T) # Hint - use numpy , pandas is MUCH SLOWER   (df.corr() )
print('Calculated correlation matrix. Shape: ',corr_matr.shape,'  seconds passed:' , np.round( time.time() - t0,1) )

print()
print(np.min(corr_matr ), 'minimal correlation' )
corr_matr_abs = np.abs( corr_matr )
print(np.mean(corr_matr_abs ), 'average absolute correlation' )
print(np.median(corr_matr_abs), 'median absolute correlation' )
print(np.min(corr_matr_abs ), 'min absolute correlation' )
print(np.std(corr_matr_abs ), 'std absolute correlation' )
print()

v = corr_matr.flatten()
plt.figure(figsize=(14,8))
t0 = time.time()
plt.hist(v, bins = 50)
plt.title('correlation coefficients distribution')
plt.show()
print(time.time() - t0, 'seconds passed')

print(np.min(corr_matr ), 'minimal correlation' )
print(np.mean(corr_matr_abs ), 'average absolute correlation' )
print(np.median(corr_matr_abs), 'median absolute correlation' )
print(np.min(corr_matr_abs ), 'min absolute correlation' )
print(np.std(corr_matr_abs ), 'std absolute correlation' )
for t in [0.5,0.6, 0.7,0.8,0.9,0.95,0.97,0.98,.99]:
    print( ((np.abs(v) < 0.99999999) & (np.abs(v) > t)).sum()/2 , 'number of pairs correlated more than', t  )
v.shape

print()
print()
for threshold_corr in [0.99,0.98,00.97,0.95,0.93,0.90,0.85,0.8,0.75,0.7,0.65,0.6,0.55,.5,0.45,0.4,0.35,0.3]: 
    a,b = np.where( np.triu(np.abs(corr_matr),1) > threshold_corr )
    if len(a) > 50: break
print('for threshold:', threshold_corr,'we have ',len(a),' correlated pairs' )


d = pd.DataFrame()
for i in range(len(a)):
    Iai, Ibi =  I[a[i]], I[b[i]]
    d.loc[i,'Gene1'] = adata[mask].var.index[Iai] 
    d.loc[i,'Gene2'] = adata[mask].var.index[Ibi ] 
    d.loc[i,'Correlation'] = corr_matr[a[i],b[i]] 
    d.loc[i,'Correlation Abs'] = np.abs(d.loc[i,'Correlation'] )
    d.loc[i,'Gene1 Expression Sum'] = np.asarray(adata[mask].X[:,Iai].sum(axis = 0)).ravel()[0]
    d.loc[i,'Gene2 Expression Sum'] = np.asarray(adata[mask].X[:,Ibi].sum(axis = 0)).ravel()[0]
    
print(time.time() - t0, 'seconds passed')
    
display( d.sort_values('Correlation Abs',ascending = False).head(30) )


#clustermap
import seaborn as sns
t0 = time.time()
sns.clustermap(np.abs(corr_matr[:3000,:3000]),cmap='vlag');
print( np.round(time.time()- t0,1), ' seconds passed.')



In [ ]:
t0=time.time()
w,v = np.linalg.eig(corr_matr[:2000,:2000])
print(time.time()-t0, 'seconds passed')
w = np.real(w.ravel())
w = np.sort(w)[::-1]
plt.plot(w,'*-')
plt.title('Eigenvalues of correlation matrix ')
plt.show()

# Visualizatios

## Compare PCA before and after pooling 

In [ ]:
mask = np.ones(len(adata)).astype(bool)

for color_by_mode in  ['cell_cycle_phase', 'n.umi', 'pct_counts_mt']:
    if color_by_mode not in adata.obs:
        color_by_mode = None
    color_by_field_name = color_by_mode

    fig = plt.figure(figsize = (25,12))
    plt.suptitle(str_data_inf + ' PCA before pooling and after.  n_cells: ' + str(mask.sum()), fontsize = 20   )#' 
    
    c = 0; 
    #for col in ['cell_type', 'n.umi', 'pct_counts_mt']:
    if 1:
        c+=1; fig.add_subplot(1, 2 , c)         
        #color_by = adata[mask].obs.drug #color_by = adata[mask].obs.phase
        #color_by = (adata[mask].obs[col])
        #color_by = (adata.obs['product_name'])

        if color_by_mode is None:
            ax = sns.scatterplot(x=X_pca1[:,0],y=X_pca1[:,1])# ,   hue= color_by)#, alpha = 0.7, marker = '.', legend = None)
        else:
            color_by = (adata.obs[color_by_field_name])
            ax = sns.scatterplot(x=X_pca1[:,0],y=X_pca1[:,1] ,   hue= color_by, palette = "viridis")# sns.color_palette("viridis") )#, alpha = 0.7, marker = '.', legend = None)
            plt.setp(ax.get_legend().get_texts(), fontsize='20') # for legend text
            plt.setp(ax.get_legend().get_title(), fontsize='20') # for legend title

        #plt.setp(ax.get_legend().get_texts(), fontsize='15') # for legend text
        #plt.setp(ax.get_legend().get_title(), fontsize='15') # for legend title
        plt.title('PCA without pooling. (ALL genes)', fontsize=20 )
        plt.xlabel('PCA1', fontsize=20 )
        plt.ylabel('PCA2', fontsize=20 )
        c+=1; fig.add_subplot(1, 2 , c)         
        #color_by = adata[mask].obs.drug #color_by = adata[mask].obs.phase
        #color_by = (adata.obs['product_name'])
        if color_by_mode is None:
            ax = sns.scatterplot(x=X_pca[:,0],y=X_pca[:,1])# ,   hue= color_by)#, alpha = 0.7, marker = '.', legend = None)
        else:
            color_by = (adata.obs[color_by_field_name])
            #color_by = (adata.obs[color_by_field_name]) > np.median( adata.obs[color_by_field_name].values ) 
            ax = sns.scatterplot(x=X_pca[:,0],y=X_pca[:,1] ,   hue= color_by, palette = "viridis")# sns.color_palette("viridis") )#, alpha = 0.7, marker = '.', legend = None)
            plt.setp(ax.get_legend().get_texts(), fontsize='20') # for legend text
            plt.setp(ax.get_legend().get_title(), fontsize='20') # for legend title
        #plt.setp(ax.get_legend().get_texts(), fontsize='15') # for legend text
        #plt.setp(ax.get_legend().get_title(), fontsize='15') # for legend title
        plt.title('PCA WITH pooling. (ALL genes)', fontsize=20 )
        plt.xlabel('PCA1', fontsize=20 )
        plt.ylabel('PCA2', fontsize=20 )

## PCA plots in several axes

In [ ]:
mask = np.ones(len(adata)).astype(bool)
from sklearn.decomposition import PCA
pca1 = PCA(n_components=5)

list_genes = list(  adata.var.index )
I = np.where(adata.var.index.isin(list_genes ) )[0]
r = pca1.fit_transform(adata[mask].X[:,I])
        
for color_by_mode in  ['n.umi']:# 'n.umi']: # , 'pct_counts_mt']:
    if color_by_mode not in adata.obs:
        color_by_mode = None
    color_by_field_name = color_by_mode
        

    c = 0
    for i,j in [(0,1), (0,2),(1,2),(0,3),(1,3),(2,3)]:
        if c % 3 == 0:
            fig = plt.figure(figsize = (25,12))
            plt.suptitle(str_data_inf + ' Change  PCA axes.  PCA from all genes.  n_cells: '\
                 +str(mask.sum()) + ' n_genes: ' + str(len(I))  , fontsize = 20 )   #' 
            c = 0; 
        
        c+=1; fig.add_subplot(1, 3 , c)
        if color_by_mode is None:
            ax = sns.scatterplot(x=r[:,i],y=r[:,j])# ,   hue= color_by)#, alpha = 0.7, marker = '.', legend = None)
        else:
            color_by = (adata.obs[color_by_field_name])
            ax = sns.scatterplot(x=r[:,i],y=r[:,j] ,   hue= color_by, palette ="viridis")#  sns.color_palette("viridis", as_cmap=True) )#, alpha = 0.7, marker = '.', legend = None)
            plt.setp(ax.get_legend().get_texts(), fontsize='20') # for legend text
            plt.setp(ax.get_legend().get_title(), fontsize='20') # for legend title
        plt.xlabel('PCA'+str(i+1),  fontsize = 20)
        plt.ylabel('PCA'+str(j+1),  fontsize = 20)
        plt.title('PCA: '+ str(i+1) +' '+str(j+1),  fontsize = 20)
        

## UMAP

In [ ]:
import scipy
import umap 
t0 = time.time()
reducer = umap.UMAP()
r = reducer.fit_transform(adata.X)

fig = plt.figure(figsize = (25,12))
plt.title(str_data_inf + ' UMAP n_cells: ' + str(adata.X.shape[0]) +\
            ' n_genes: ' + str(adata.X.shape[1])  + ' '  , fontsize = 20   )#' 
if 'n.umi' not in adata.obs.columns: 
    ax = sns.scatterplot(x=v1, y = v2)#,  hue= color_by)#, palette = color_palette )#, palette = "viridis")# sns.color_palette("viridis", as_cmap=True),
                        #)# ,   alpha = 0.8, marker = '.')#, )#, legend=None)
else:
    color_by = adata.obs['n.umi'].values
    ax = sns.scatterplot(x=r[:,0], y = r[:,1],  hue= color_by)#, palette = color_palette )#, palette = "viridis")# sns.color_palette("viridis", as_cmap=True),
                        #)# ,   alpha = 0.8, marker = '.')#, )#, legend=None)
    plt.setp(ax.get_legend().get_texts(), fontsize='20') # for legend text
    plt.setp(ax.get_legend().get_title(), fontsize='20') # for legend title

plt.xlabel('UMAP1' , fontsize = 20 )
plt.ylabel('UMAP2' , fontsize = 20 )
plt.show()
print( np.round( time.time() - t0,1) , ' seconds passed ' ) 

In [ ]:
import scipy
import umap 
t0 = time.time()
n_neighbors = 50
min_dist = 0.7
reducer = umap.UMAP(n_neighbors=n_neighbors, min_dist=min_dist,)
r = reducer.fit_transform(adata.X)

fig = plt.figure(figsize = (25,12))
plt.title(str_data_inf + ' UMAP('+ str(n_neighbors)+','+str( min_dist )+ ') n_cells: ' + str(adata.X.shape[0]) +\
            ' n_genes: ' + str(adata.X.shape[1])  + ' '  , fontsize = 20   )#' 
if 'n.umi' not in adata.obs.columns: 
    ax = sns.scatterplot(x=v1, y = v2)#,  hue= color_by)#, palette = color_palette )#, palette = "viridis")# sns.color_palette("viridis", as_cmap=True),
                        #)# ,   alpha = 0.8, marker = '.')#, )#, legend=None)
else:
    color_by = adata.obs['n.umi'].values
    ax = sns.scatterplot(x=r[:,0], y = r[:,1],  hue= color_by)#, palette = color_palette )#, palette = "viridis")# sns.color_palette("viridis", as_cmap=True),
                        #)# ,   alpha = 0.8, marker = '.')#, )#, legend=None)
    plt.setp(ax.get_legend().get_texts(), fontsize='20') # for legend text
    plt.setp(ax.get_legend().get_title(), fontsize='20') # for legend title

plt.xlabel('UMAP1' , fontsize = 20 )
plt.ylabel('UMAP2' , fontsize = 20 )
plt.show()
print( np.round( time.time() - t0,1) , ' seconds passed ' ) 

## Plots in genes axes

In [ ]:
mask = np.ones( adata.X.shape[0]).astype(bool) # 

gene1 = 'E2F1' 
gene2 = 'FOXM1'
n_x_subplots = 4

c = 0 
for gene1,gene2 in [ ('E2F1','FOXM1'), ('PCNA','TOP2A'), ('CCNB1', 'CCNB2'), ('CCNE1', 'CCNB1'),
                   ('E2F1','PCNA'),('FOXM1','PCNA'), ('E2F1','TOP2A'),('FOXM1','TOP2A'), ]: 
    #,'PCNA','TOP2A', 'CCNE1', 'CCNE2', 'CDK2', 'CCNB1','CCNB2','CCNB3','CCNA2',  ]: # , 'pct_counts_mt']:
    if gene1 not in adata.var.index : continue
    if gene2 not in adata.var.index : continue
        
    color_by_mode = 'n.umi' # ,#'cell_cycle_phase', #  'PCA1ALL', 'n.umi', 'threshold_binarize_5_PCA1ALL',
    if color_by_mode in adata.obs:
        color_by_field_name = color_by_mode
        color_by = (adata[mask].obs[color_by_field_name])
    else:
        color_by_mode = None
    
    if c % n_x_subplots == 0:
        fig = plt.figure(figsize = (20,6) ); c = 0
        plt.suptitle(str_data_inf + ' Genes axes   n_cells: ' + str(adata.X.shape[0]) +\
            ' n_genes: ' + str(adata.X.shape[1])  + ' color by '+ str(color_by_mode) , fontsize = 20   )#' 
        c = 0
        
    c += 1; fig.add_subplot(1,n_x_subplots ,c)
    plt.title(gene1 + ' ' + gene2) # str(color_by_mode) + ' PCA '+str(i)+','+str(j))

    plt.legend()
    I_gene = np.where(adata.var.index == gene1)[0]
    v = adata[mask].X[:,I_gene]
    if scipy.sparse.issparse(adata.X):
        v = np.asarray(v.toarray()).ravel()
    else:
        v = np.asarray(v).ravel()
        
    v1 = v
    I_gene = np.where(adata.var.index == gene2)[0]
    v = adata[mask].X[:,I_gene]
    if scipy.sparse.issparse(adata.X):
        v = np.asarray(v.toarray()).ravel()
    else:
        v = np.asarray(v).ravel()
    v2 = v


    if color_by_mode is None:
        ax = sns.scatterplot(x=v1,y=v2)# ,   hue= color_by)#, alpha = 0.7, marker = '.', legend = None)
    else:
        ax = sns.scatterplot(x=v1,y=v2,   hue= color_by)#, alpha = 0.7, marker = '.', legend = None)
        plt.setp(ax.get_legend().get_texts(), fontsize='15') # for legend text
        plt.setp(ax.get_legend().get_title(), fontsize='15') # for legend title    plt.xlabel('PCA1' , fontsize = 20 )
        ax.legend().set_visible(False)

    plt.xlabel(gene1,  fontsize = 20)
    plt.ylabel(gene2,  fontsize = 20)
plt.show()  

In [ ]:
print(np.round(time.time()-t0start,1),  np.round( (time.time()-t0start)/60,1),  np.round( (time.time()-t0start)/3600,1), 
      'seconds, munutes, hours  passed')